# Adversarial attacks with FGSM (Fast Gradient Sign Method) 

The Fast Gradient Sign Method (FGSM) is a simple yet effective method to generate adversarial images. First introduced by Goodfellow et al. in their paper, Explaining and Harnessing Adversarial Examples, FGSM works by:

1. Taking an input image
2. Making predictions on the image using a trained CNN
3. Computing the loss of the prediction based on the true class label
4. Calculating the gradients of the loss with respect to the input image
5. Computing the sign of the gradient
6. Using the signed gradient to construct the output adversarial image

> FGSM computes the gradients of a loss function (e.g., mean-squared error or categorical cross-entropy) with respect to the input image and then uses the sign of the gradients to create a new image (i.e., the adversarial image) that **maximizes** the loss.

![](https://929687.smushcdn.com/2633864/wp-content/uploads/2021/02/fgsm_equation.png?lossy%3D1%26strip%3D1%26webp%3D1)

In [1]:
from pyimagesearch.simplecnn2 import SimpleCNN
from pyimagesearch.fgsm import generate_image_adversary_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2

### Get the dataset

In [2]:
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX/255.0
testX = testX/255.0

In [3]:
#Since is a black and white image, expand it's dimentions
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

In [4]:
#One hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

### Get the model

In [5]:
from pickletools import optimize

model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy",optimizer= Adam(learning_rate=1e-3), metrics=["accuracy"])

2022-09-22 20:06:07.318610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 20:06:07.326268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 20:06:07.326476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 20:06:07.327049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
#Train the model on the images (step 2)
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=10, verbose=1)

Epoch 1/10


2022-09-22 20:06:09.326371: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-09-22 20:06:09.585200: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-22 20:06:09.586144: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-22 20:06:09.586190: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-09-22 20:06:09.587126: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-22 20:06:09.587237: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


938/938 [==============================] - 6s 4ms/step - loss: 0.2279 - accuracy: 0.9311 - val_loss: 0.0653 - val_accuracy: 0.9777
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0892 - accuracy: 0.9734 - val_loss: 0.0477 - val_accuracy: 0.9841
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0667 - accuracy: 0.9787 - val_loss: 0.0465 - val_accuracy: 0.9851
Epoch 4/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0564 - accuracy: 0.9822 - val_loss: 0.0403 - val_accuracy: 0.9860
Epoch 5/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0467 - accuracy: 0.9849 - val_loss: 0.0367 - val_accuracy: 0.9863
Epoch 6/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0413 - accuracy: 0.9876 - val_loss: 0.0374 - val_accuracy: 0.9885
Epoch 7/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0361 - accuracy: 0.9887 - val_loss: 0.0347 - val_accuracy: 0.9888
Epoch 8/10
938/93

### Step2: Making predictions on the image using a trained CNN

In [7]:
(loss, acc) = model.evaluate(x=testX, y=testY,verbose=0)
print("Loss: {:.4f},\nAccuracy: {:.4f}".format(loss, acc))

Loss: 0.0350,
Accuracy: 0.9867


### Step 3 - 6: Done by the generate_image_adversary function and the following code

In [9]:
for i in np.random.choice(np.arange(0, len(testX)), size=(10,)):
    #Grab the current image and label
    image = testX[i]
    label = testY[i]

    #generate adversary image from the current image and make prediction from that adversary
    adversary = generate_image_adversary_model(model, image.reshape(1,28,28,1), label, eps=0.1)
    pred = model.predict(adversary)

    #Scale the image and adversary to the [0,255]scale an convert them to 8bit due to cv2
    adversary = adversary.reshape((28,28))*255
    adversary = np.clip(adversary, 0, 255).astype("uint8") #and clip the values that exceed
    image = image.reshape((28,28))*255
    image = image.astype("uint8")

    #conver images to 3 channels so we can draw the labels on them
    image = np.dstack([image]*3)
    adversary = np.dstack([adversary]*3)

    #resize the images for better visualization
    image = cv2.resize(image, (96,96))
    adversary = cv2.resize(image, (96,96))

    #Get the predicted labels for both the original image and the adversarial one
    imagePred = label.argmax()
    adversarypred = pred[0].argmax()
    color = (0, 0, 255)

    print("original image label: {} \nAdversary label: {}".format(imagePred, adversarypred))

    #if image prediction doesn't match with current original image label, set color to red
    if imagePred != adversarypred:
        color=(0,0,255)
    #draw the poredictions on the respecive output images
    cv2.putText(image, str(imagePred), (2,25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0,255,0), 2)
    cv2.putText(adversary, str(adversarypred), (2,25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)

    output = np.hstack([image, adversary])
    cv2.imshow("FGSM adversarial images", output)
    cv2.waitKey(0)

cv2.destroyAllWindows()



1/1 [==============================] - 0s 16ms/step
original image label: 5 
Adversary label: 3


QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to target thread (0x563989bf0d30)

QObject::moveToThread: Current thread (0x563989bf0d30) is not the object's thread (0x5639886cbfe0).
Cannot move to tar

1/1 [==============================] - 0s 14ms/step
original image label: 2 
Adversary label: 3
1/1 [==============================] - 0s 15ms/step
original image label: 4 
Adversary label: 7
1/1 [==============================] - 0s 16ms/step
original image label: 1 
Adversary label: 4
1/1 [==============================] - 0s 14ms/step
original image label: 6 
Adversary label: 0
1/1 [==============================] - 0s 16ms/step
original image label: 0 
Adversary label: 6
1/1 [==============================] - 0s 15ms/step
original image label: 5 
Adversary label: 3
1/1 [==============================] - 0s 14ms/step
original image label: 2 
Adversary label: 8
1/1 [==============================] - 0s 15ms/step
original image label: 3 
Adversary label: 7
1/1 [==============================] - 0s 15ms/step
original image label: 1 
Adversary label: 4
